# プログラミング勉強会
## 2022/11/18

pythonに触ったことのない、基礎文法がわからない人にはこれがおすすめです

[![youtube](https://img.youtube.com/vi/HyU3XL2F9GE/0.jpg)](https://www.youtube.com/watch?v=HyU3XL2F9GE)

---

# テーマ: スクレイピングに触れてみよう

# 1. そもそもスクレイピングってなに？

[参考: スクレイピングとは何かやさしく解説](https://www.sbbit.jp/article/cont1/71102)

簡単にいうと, **Webサイトからデータを取得すること** です.

データサイエンスの分野で言うところの **データ収集** です.

機械学習とか深層学習で大量のデータが必要だったりするときに便利な印象


## スクレイピングの注意点
データを取得する際には, **Webサイトの利用規約** を確認する必要があります.

また、負荷をかけすぎると犯罪につながるので気をつけましょう

[スクレイピング、クローリングする時の注意点](https://docs.pyq.jp/column/crawler.html)

↑**必読**↑

---

# 2. スクレイピングの流れ(BeautifulSoup使用)

1. データを取得したいWebサイトを見つける
2. HTTPリクエストを投げてWebサイトのHTMLを取得する
3. `soup`オブジェクトを作成する
4. `soup`オブジェクトからデータを取得する
5. 必要に応じてデータを整形する

---

# 3. 実際にやってみる

今回の目標はPythonに関するブログの記事を取得すること

https://zenn.dev/topics/python

## 3.0. 必要なライブラリのインストール

- `requests`: HTTPリクエストを投げるためのライブラリ
- `BeautifulSoup`: HTMLを解析するためのライブラリ
- `time`: 時間を制御するためのライブラリ, 今回はスクレイピングの間隔を調整するために使用
- `pandas`: データを整形するためのライブラリ

```python
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
```

In [31]:
# ↑を写経

# 3.1. データを取得したいWebサイトを見つける

In [32]:
url = 'https://zenn.dev/topics/python'

https://zenn.dev/robots.txt

↑を確認

# 3.2. HTTPリクエストを投げてWebサイトのHTMLを取得する

```python
# 連続でアクセスするとエラーになるので、1秒待機させる
time.sleep(1)
# HTTPリクエストを投げてWebサイトのHTMLを取得する
res = requests.get(url)
print(res.text)
```

In [33]:
#  ↑を写経（コメントは不要）

# 3.3. `soup`オブジェクトを作成する

```python
# HTMLを解析するためのsoupオブジェクトを作成する
soup = BeautifulSoup(res.text, 'html.parser')
print(soup.prettify())
```

In [34]:
# ↑を写経（コメントは不要）

# 3.4. `soup`オブジェクトからデータを取得する

### 参考資料
- [最低限覚えておきたいCSSのセレクタ](https://morizyun.github.io/web/css-selector.html)
- [図解！Python BeautifulSoupの使い方を徹底解説！](https://ai-inter1.com/beautifulsoup_1/)


```python
# cssセレクタを使ってデータを取得する
# class名がAriticleList__contentから始まる要素を取得する
article_list = soup.select('[class^=ArticleList_content]')

# 記事の情報を取得する関数
def get_article_info(article):
    # h2タグの中のテキストを取得
    title = article.select_one("h2").text
    # class名がArticleList_userNameから始まる要素の中のテキストを取得
    author_name = article.select_one("[class^=ArticleList_userName]").text
    # class名がArticleList_linkから始まる要素の中のhref属性を取得
    relative_link = article.select_one("[class^=ArticleList_link]").get("href")
    # class名がArticleList_likeから始まる要素の中のテキストを取得
    # ただし、いいねが0件の場合はNoneになるので、0に置き換える
    like = (
        article.select_one("[class^=ArticleList_like]").text
        if article.select_one("[class^=ArticleList_like]")
        else 0
    )
    # 辞書型で返す
    return {"タイトル": title, "著者": author_name, "リンク": f'https://zenn.dev{relative_link}', "いいね": int(like)}

article_info_list =  []
# article_listはイテラブルなので、for文で回す
for article in article_list:
    article_info_list.append(get_article_info(article))

print(article_info_list)
```

In [35]:
# ↑を写経（コメントは不要）

# 3.5. 必要に応じてデータを整形する

データ分析でもおなじみpandasを使います

```python
# 辞書型のリストをデータフレームに変換する
df = pd.DataFrame(article_info_list)
# いいね順にソートする
df = df.sort_values("いいね", ascending=False)
# リンクをクリックできるようにする
df.head(10).style.format({"リンク": lambda x: f'<a href="{x}">{x}</a>'}) 
```

In [36]:
# ↑を写経（コメントは不要）

---

In [37]:
# ここまでのコード
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

url = "https://zenn.dev/topics/python"
# 連続でアクセスするとエラーになるので、1秒待機させる
time.sleep(1)
# HTTPリクエストを投げてWebサイトのHTMLを取得する
res = requests.get(url)

# HTMLを解析するためのsoupオブジェクトを作成する
soup = BeautifulSoup(res.text, "html.parser")

# cssセレクタを使ってデータを取得する
# class名がAriticleList__contentから始まる要素を取得する
article_list = soup.select("[class^=ArticleList_content]")

# 記事の情報を取得する関数
def get_article_info(article):
    # h2タグの中のテキストを取得
    title = article.select_one("h2").text
    # class名がArticleList_userNameから始まる要素の中のテキストを取得
    author_name = article.select_one("[class^=ArticleList_userName]").text
    # class名がArticleList_linkから始まる要素の中のhref属性を取得
    relative_link = article.select_one("[class^=ArticleList_link]").get("href")
    # class名がArticleList_likeから始まる要素の中のテキストを取得
    # ただし、いいねが0件の場合はNoneになるので、0に置き換える
    like = (
        article.select_one("[class^=ArticleList_like]").text
        if article.select_one("[class^=ArticleList_like]")
        else 0
    )
    # 辞書型で返す
    return {
        "タイトル": title,
        "著者": author_name,
        "リンク": f"https://zenn.dev{relative_link}",
        "いいね": int(like),
    }


article_info_list = []
# article_listはイテラブルなので、for文で回す
for article in article_list:
    article_info_list.append(get_article_info(article))

# 辞書型のリストをデータフレームに変換する
df = pd.DataFrame(article_info_list)
# いいね順にソートする
df = df.sort_values("いいね", ascending=False)
df.head(10).style.format({"リンク": lambda x: f'<a href="{x}">{x}</a>'})

,タイトル,著者,リンク,いいね
39,話題のStable Diffusionがオープンソース化されたのでローカルで動かしてみる,koyoarai_,https://zenn.dev/koyoarai_/articles/02f3ed864c6127bb2049,339
44,中級者へのModern Python,ganyariya,https://zenn.dev/ganariya/articles/intermediate-python,170
47,テーブルデータ向けの自然言語特徴抽出術,koukyo1994,https://zenn.dev/koukyo1994/articles/9b1da2482d8ba1,112
41,実体のないマウスを作りました,Yuki,https://zenn.dev/ninzin/articles/94b05fdb9edf53,106
43,【Python入門】ネコでも分かる「Pythonの基本」まとめ,NekoAllergy,https://zenn.dev/nekoallergy/articles/0adf4d57acf9b4,97
45,botterのためのasyncio,まちゅけん,https://zenn.dev/mtkn1/articles/c61e77c1d221aa,96
40,pathlibはいいぞ,alivelimb,https://zenn.dev/alivelimb/articles/0f3f8d61d91d57,77
46,コーディングテストに必要なアルゴリズムを図でやさしく説明してみた！,yutohub,https://zenn.dev/yutohub/articles/b53c20a6e6a9bc,71
26,matplotlibの備忘録,canard0328,https://zenn.dev/canard0328/articles/memorandum-of-matplotlib,32
0,続・MeCabの分かち書きを並列処理で高速化する,hpp,https://zenn.dev/hpp/articles/64466d9476fb2b,28


---

# お疲れ様でした！
ここまででひとまずスクレイピングの基本的な流れを理解できたと思います

余力がある人は、以下の「関数化してページネーション対応」をみて自分で実装してみてください

---

## 関数化してページネーション対応

処理を分けて関数化することにより、使い回しが効く。

それだけでなくカプセル化としての役割もあることから、予期せぬエラーを防ぐことができる。

また、全体を見たとき、処理の流れがわかりやすくなる。

In [38]:
def get_soup(url: str) -> BeautifulSoup:
    """
    urlからBeautifulSoupオブジェクトを取得する
    
    Parameters
    ----------
    url : str
        
    Returns
    -------
    soup : BeautifulSoup
    """
    time.sleep(1)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_articles(soup: BeautifulSoup) -> list:
    """
    soupから記事の情報を取得する

    Parameters
    ----------
    soup : BeautifulSoup

    Returns
    -------
    article_list : list[BeautifulSoup]
    """
    article_list = soup.select('[class^=ArticleList_content]')
    return article_list

def get_article_info(article: BeautifulSoup) -> dict:
    """
    articleから記事の情報を取得する

    Parameters
    ----------
    article : BeautifulSoup

    Returns
    -------
    article_info : dict
    """
    title = article.select_one("h2").text
    author_name = article.select_one("[class^=ArticleList_userName]").text
    relative_link = article.select_one("[class^=ArticleList_link]").get("href")
    like = (
        article.select_one("[class^=ArticleList_like]").text
        if article.select_one("[class^=ArticleList_like]")
        else 0
    )
    return {"タイトル": title, "著者": author_name, "リンク": f'https://zenn.dev{relative_link}', "いいね": int(like)}

def get_article_info_df(url):
    """
    urlから記事の情報を取得してデータフレームにする

    Parameters
    ----------
    url : str

    Returns
    -------
    df : pd.DataFrame
    """
    soup = get_soup(url)
    article_list = get_articles(soup)
    article_info_list =  []
    for article in article_list:
        article_info_list.append(get_article_info(article))
    df = pd.DataFrame(article_info_list)
    return df

# ここ大事ポイント
```python
def get_article_info_df(url):
    # 1.soupオブジェクトを作成する
    soup = get_soup(url)
    # 2.記事の情報のsoupオブジェクトのリストを取得する
    article_list = get_articles(soup)
    # 3.soupのリストをfor文で回して、記事の情報を取得する
    article_info_list =  []
    for article in article_list:
        article_info_list.append(get_article_info(article))
    # 4.辞書型のリストをデータフレームに変換する
    df = pd.DataFrame(article_info_list)
    return df
```
関数化したおかげで、処理の流れがわかりやすくなる。

beautifulsoupでのスクレイピングが4ステップにまとめられた。

In [39]:
# ページ数を指定して、記事の情報を取得する
def get_zenn_article_df_by_topic(max_page, topic="python"):
    # ページが変わるとURLが変わる(URLクエリパラメータという)
    urls = [f"https://zenn.dev/topics/{topic}?page={i}" for i in range(1, max_page + 1)]

    df_list = []
    for url in urls:
        try:
            df = get_article_info_df(url)
            df_list.append(df)
            print(f"success: {url}")
        except:
            print(f"error!: {url}の取得に失敗しました")
    df = pd.concat(df_list)
    return df

In [40]:
# 3ページ分取ってきてもらう
df = get_zenn_article_df_by_topic(max_page=3)

success: https://zenn.dev/topics/python?page=1
success: https://zenn.dev/topics/python?page=2
success: https://zenn.dev/topics/python?page=3


In [41]:
print(df.shape)
df.tail()

(144, 4)


,タイトル,著者,リンク,いいね
43,線形代数と確率過程の話,ロボ太,https://zenn.dev/kaityo256/articles/markov_eig...,37
44,持ち家と賃貸の金銭面の比較 with Python in 23区,xiongjie,https://zenn.dev/xiongjie/articles/09eddca747fe05,34
45,この素晴らしいJuliaにDeep Learningを！,skypenguins,https://zenn.dev/skypenguins/articles/b44bff12...,42
46,その日のコミット数に応じて増える Docker キャラクターのコンテナ,ogty,https://zenn.dev/ogty/articles/2eed4d7a-d1de-4...,21
47,機械学習の高速化テクニック3選,yagiyuki,https://zenn.dev/yagiyuki/articles/accelerate_ml,23


In [42]:
# トピックを指定して、記事の情報を取得する
df = get_zenn_article_df_by_topic(max_page=3, topic="データサイエンス")
df.head()

success: https://zenn.dev/topics/データサイエンス?page=1
success: https://zenn.dev/topics/データサイエンス?page=2
success: https://zenn.dev/topics/データサイエンス?page=3


,タイトル,著者,リンク,いいね
0,統計検定準1級に受かるための勉強法・参考サイト,でるぶ,https://zenn.dev/dlbsabu/articles/5300da50921070,85
1,【資格】統計初心者が統計検定２級・準１級を合計２か月で合格した話,yanagikk,https://zenn.dev/yanagikk/articles/a32a575a9163e2,57
2,Kaggleで金融コンペを開催するための(僕が知っている)すべて,tomo@Alpaca,https://zenn.dev/gamella/articles/eaf7fe5a96bdf0,56
3,「プログラミング未経験者がWeb/ITエンジニアに転職するためには何をしたらいいのか？」を統...,堀口セイト,https://zenn.dev/seito/articles/8f5ce6bee847c2,32
4,PythonによるDomain Adaptation Learnerの実装,Shingo Uto,https://zenn.dev/s1ok69oo/articles/a319078f8c6af1,21


In [46]:
df = get_zenn_article_df_by_topic(max_page=10, topic="python")

success: https://zenn.dev/topics/python?page=1
success: https://zenn.dev/topics/python?page=2
success: https://zenn.dev/topics/python?page=3
success: https://zenn.dev/topics/python?page=4
success: https://zenn.dev/topics/python?page=5
success: https://zenn.dev/topics/python?page=6
success: https://zenn.dev/topics/python?page=7
success: https://zenn.dev/topics/python?page=8
success: https://zenn.dev/topics/python?page=9
success: https://zenn.dev/topics/python?page=10


In [47]:
# いいねの多い順にソートする
df.sort_values('いいね', ascending=False).head(10).style.format({'リンク': lambda x: f'<a href="{x}">{x}</a>'})

,タイトル,著者,リンク,いいね
11,君には今から3時間で機械学習Webアプリを作ってもらうよ,alivelimb,https://zenn.dev/alivelimb/articles/20220528-streamlit-ml-app,427
39,話題のStable Diffusionがオープンソース化されたのでローカルで動かしてみる,koyoarai_,https://zenn.dev/koyoarai_/articles/02f3ed864c6127bb2049,339
12,Python互換の静的型付け言語「Erg」,shiba,https://zenn.dev/mtshiba/articles/a38c9fcd9646d4,339
17,【必見】プログラマーが学習・開発で絶対に登録するべき15のYouTubeチャンネル,Shota Nukumizu,https://zenn.dev/nameless_sn/articles/recommended_youtube,247
32,Pythonでの開発・CI/CDの私的ベストプラクティス2022,dajiaji,https://zenn.dev/dajiaji/articles/e29005502a9e78,171
44,中級者へのModern Python,ganyariya,https://zenn.dev/ganariya/articles/intermediate-python,170
5,2021年Python開発リンター導入のベストプラクティス,Yusuke,https://zenn.dev/yhay81/articles/yhay81-202102-pythonlint,145
14,データ収集から機械学習まで全て行って競馬の予測をしてみた,kami,https://zenn.dev/kami/articles/66e400c8a43cd08a5d7f,142
30,Pythonで自分だけのクソライブラリを作る方法,karaage0703,https://zenn.dev/karaage0703/articles/db8c663640c68b,122
3,私が考えるLambda開発環境のベストプラクティス,Dai@FastLabel,https://zenn.dev/faycute/articles/9024115c4241b8,122


## ここまで頑張れた人はすごい！！

さらに頑張る人へのおすすめ教材

- [Youtube-【Webスクレイピング超入門】2時間で基礎を完全マスター！PythonによるWebスクレイピング入門 連結版](https://www.youtube.com/watch?v=VRFfAeW30qE)

↑こちらの動画は、スクレイピングの基礎を学ぶのに最適な動画です。色々省略した部分もあるので是非みてほしい

- [図解！PythonでSeleniumを使ったスクレイピングを徹底解説！(インストール・使い方・Chrome)](https://ai-inter1.com/python-selenium/)
  
↑別のライブラリを使ったスクレイピング(クローリング)の方法を学べる

- [HTMLとCSSの基本をサルでもわかるようにまとめた【入門・初心者向け】](https://www.ituore.com/entry/html-css-basic)

↑そもそもHTMLとCSSの基本を学びたい人はこちら